# Setup libraries

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Parameters

In [57]:
verbose_mode = False # show intermediate results
# ---------------------------------------------------------------------------

# ** image transformation thresholds **
sobel_ksize = 3 # Choose a larger odd number to smooth gradient measurements
hls_channels_param = [False, True,False] # which channels to use for HLS transform
rgb_channels_param = [True, True, False] # which channels to use for RGB transform
gradient_channel_param = False # use gradient transform or not

# ** masks and warping **
warp_source_coordinates = [[150, 720] , [570, 460], [710, 460], [1175, 720]]
warp_destination_coordinates = [[250,  720], [250,    0],  [1065,   0], [1065, 720]]


# ** pixel to real-world conversion **
xm_per_pix = 3.7/700 #pixels to real-world

# ** sanity checks **
max_width_std = 0.1 #max std of length width across all y coords

max_lane_width = 4.5 #max width in meters (mean)
min_lane_width = 3.4 #max width in meters (mean)

bottom_max_lane_width = 4.3 #max width in meters (bottom)
bottom_min_lane_width = 3.6 #max width in meters (bottom)

not_detected_cnt_thres = 4 #if lane not detected with poly approx method in x frames revert back to histogram method
not_detected_cnt_max_thres = 100 #force new poly after x frames even if current frame giving bad results (this is to avoid being locked with one function all the time) 


# Preparation

In [12]:
# 1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

# images - list of images for calibration
# xdim - count of squares horizontally
# ydim - count of squares vertically
def calibrateCammera(images, xdim = 9, ydim = 6, test = False):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((ydim*xdim,3), np.float32)
    objp[:,:2] = np.mgrid[0:xdim, 0:ydim].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (xdim,ydim), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            #testing purpose only
            if (test):
                cv2.drawChessboardCorners(img, (xdim,ydim), corners, ret)
                write_name = 'camera_cal_out/corners_found'+str(idx)+'.jpg'
                cv2.imwrite(write_name, img)
                plt.imshow(img)
                plt.title('Camera Calibration Example Image', fontsize=20)
    
    # Do camera calibration given object points and image points
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    
    if (test):
        #undistrot demo
        for idx, fname in enumerate(images):
            img = cv2.imread(fname)
            undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
            write_name = 'camera_cal_out/undistorted_chessboard'+str(idx)+'.jpg'
            cv2.imwrite(write_name, undistorted_img)
    return mtx, dist

def undistort_and_warp(image, mtx, dist):
    undistored_img = cv2.undistort(image, mtx, dist, None, mtx)
    return warp(undistored_img)

def gaussian_blur(image, kernel=5):
    output = cv2.GaussianBlur(image, (kernel,kernel), 0)
    return output

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    
#     mask = np.zeros_like(img)
#     region_of_interest_vertices = np.array(corners_src, dtype=np.int32)
#     cv2.fillPoly(mask, [region_of_interest_vertices], 1)
#     img = cv2.bitwise_and(img, mask)
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 1
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, [np.array(vertices, dtype=np.int32)], 1) #ignore_mask_color
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

#    image = undistored_img = cv2.undistort(img, mtx, dist, None, mtx)
def warp(image, image_color = None, warp_source_coords = None, warp_dest_coords = None, test = False):
    img = image.copy()
    img_size = (image.shape[1], image.shape[0])
    #1. get corners for transformation 
    #source image corners (hardcoded, picked from one image, applied to all other assumign same camera pos)
    if (warp_source_coords is None):
        warp_source_coords = [[150, 720] , [570, 460], [710, 460], [1175, 720]]
    # c) define 4 destination points
    if (warp_dest_coords is None):
        warp_dest_coords = [[250,  720], [250,    0],  [1065,   0], [1065, 720]]
        
    corners_src = np.float32(warp_source_coords)
    corners_target = np.float32(warp_dest_coords) 
    
    # 2) Convert to grayscale
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # apply region of interest mask (same as warped area)
    mask_vertices = corners_src.copy()
    mask_vertices[0][0] = mask_vertices[0][0] - 50
    mask_vertices[1][0] = mask_vertices[1][0] - 50
    mask_vertices[2][0] = mask_vertices[2][0] + 200
    mask_vertices[3][0] = mask_vertices[3][0] + 200
    img = region_of_interest(img , mask_vertices)    
    
    # 2. Get transformation matrix based on corners
    M = cv2.getPerspectiveTransform(corners_src, corners_target)
    Minv = cv2.getPerspectiveTransform(corners_target, corners_src)
    
    # 3. Perform transformation 
    # e) use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR) 
    warped = warped *255
    warped = warped.astype('uint8')
    #show 
    if (test):
        img = np.dstack(( img.astype('uint8'), img.astype('uint8'), img.astype('uint8'))) * 255 #change b&w mask to rgb
        #mark the warp src points
        pts = np.array(warp_source_coords, np.int32) 
        pts = pts.reshape((-1, 1, 2))  
        # Using cv2.polylines() method 
        img = cv2.polylines(img, [pts], isClosed = True, color = (255, 0, 0), thickness = 2)
        #mark the dest points
        pts_d = np.array(warp_dest_coords, np.int32) 
        pts_d = pts_d.reshape((-1, 1, 2))  
        # Using cv2.polylines() method 
        warped_overlay = np.dstack(( warped.astype('uint8'), warped.astype('uint8'), warped.astype('uint8'))) #change b&w mask to rgb
        warped_overlay = cv2.polylines(warped_overlay, [pts_d], isClosed = True, color = (255, 0, 0), thickness = 2)
        
        # plot images
        if (image_color is not None):
            #warp color image
            image_color = cv2.warpPerspective(image_color, M, img_size, flags=cv2.INTER_LINEAR) 
            image_color = cv2.polylines(image_color, [pts_d], isClosed = True, color = (255, 0, 0), thickness = 2)
            f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
            f.tight_layout()
            ax1.imshow(img)
            ax1.set_title('Unwarped, src points', fontsize=30)
            # -
            ax1_5.imshow(image_color)
            ax1_5.set_title('Warped color', fontsize=30)
            # -
            ax2.imshow(warped_overlay, cmap='gray')
            ax2.set_title('Warped gray', fontsize=30)
            plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        else:
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
            f.tight_layout()
            ax1.imshow(img)
            ax1.set_title('Unwarped, src points', fontsize=30)
            # -
            ax2.imshow(warped, cmap='gray')
            ax2.set_title('Warped gray', fontsize=30)
            plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    return warped, M, Minv

#
# undistorted_img - undistored color image
# warped - warped lane overlay obtained during lane pixel / curve calculation
# Minv - inverse transform for warping 'wraped image'
def unwarp(undistorted_img, overlay_warped, Minv):
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(overlay_warped, Minv, (undistorted_img.shape[1], undistorted_img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted_img, 1, newwarp, 0.3, 0)
    return result


# Functions relevant to steps in the pipeline

## Image transformations (gradient, color transform etc.)

In [60]:
# sobel filter

def sobel_abs_thresh_transform(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the derivative or gradient
    sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    sobel = np.uint8(255*sobel/np.max(sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(sobel)
    binary_output[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    return binary_output

def sobel_magnitude_thresh_transform(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    sobel = np.uint8(255*magnitude/np.max(magnitude))
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(sobel)
    binary_output[(sobel >= mag_thresh[0]) & (sobel <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    return binary_output

def sobel_direction_thresh_transform(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    sobelx = np.absolute(sobelx)
    sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    gradient = np.arctan2(sobely, sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(gradient)
    binary_output[(gradient >= thresh[0]) & (gradient <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    return binary_output

def sobel_thresh_transform(image, kernel_size=3, sobel_abs_thresh_x=(40, 100), sobel_abs_thresh_y=(40, 100), sobel_magnitude_thresh=(30, 100), sobel_direction_thresh=(0.7, 1.3)):
    # Apply each of the thresholding functions
    gradx = sobel_abs_thresh_transform(image, orient='x', sobel_kernel=kernel_size, thresh= sobel_abs_thresh_x)
    grady = sobel_abs_thresh_transform(image, orient='y', sobel_kernel=kernel_size, thresh= sobel_abs_thresh_y)
    mag_binary = sobel_magnitude_thresh_transform(image, sobel_kernel=kernel_size, mag_thresh= sobel_magnitude_thresh)
    dir_binary = sobel_direction_thresh_transform(image, sobel_kernel=kernel_size, thresh= sobel_direction_thresh)

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    combined[((gradx == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    #plt.imshow(combined, cmap='gray')
    return(combined)

# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_thresh_transform(img, channel = [False,True,False],thresh=(0, 255), test = False):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    S = hls[:,:,2]
    binary_output4 = np.zeros_like(S)
    binary_output4[(S > thresh[0]) & (S <= thresh[1])] = 1
    
    H = hls[:,:,0] 
    h_thres = (170,200) #170, 255
    binary_output2 = np.zeros_like(H)
    binary_output2[(H > h_thres[0]) & (H <= h_thres[1])] = 1
    
    l_thres = (170,200)
    L = hls[:,:,1]
    binary_output3 = np.zeros_like(L)
    binary_output3[(L > l_thres[0]) & (L <= l_thres[1])] = 1
    
    #combine
    binary_output = np.zeros_like(H)
    binary_output[(channel[0] & binary_output2 == 1) | (channel[1] & binary_output3 == 1) | (channel[2] & binary_output4 == 1)] = 1
    
    if (test):
        #TEST

        f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(binary_output2, cmap='gray')
        ax1.set_title('HLS Thres (H)', fontsize=30)
        # -
        ax1_5.imshow(binary_output3, cmap='gray')
        ax1_5.set_title('HLS Thres (L)', fontsize=30)
        # -
        ax2.imshow(binary_output4, cmap='gray')
        ax2.set_title('HLS Thres (S)', fontsize=30)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    #END TEST
    # 3) Return a binary image of threshold result
    #binary_output = np.copy(img) # placeholder line
#    return binary_output
#    return binary_output3
    return binary_output

def rgb_transform(img, channel = [True,True,False], thres=(170,255), test = False):
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    
    r_thres = (170,200) #170, 255
    binary_output1 = np.zeros_like(R)
    binary_output1[(R > r_thres[0]) & (R <= r_thres[1])] = 1
    
    g_thres = (170,200) #170, 255
    binary_output2 = np.zeros_like(G)
    binary_output2[(G > g_thres[0]) & (G <= g_thres[1])] = 1
    
    b_thres = (170,200) #170, 255
    binary_output3 = np.zeros_like(B)
    binary_output3[(B > b_thres[0]) & (B <= b_thres[1])] = 1
    
    #combine
    binary_output = np.zeros_like(R)
    binary_output[(channel[0] & binary_output1 == 1) | (channel[1] & binary_output2 == 1) | (channel[2] & binary_output3 == 1)] = 1
    #binary_output[(binary_output1 == 1) | (binary_output2 == 1)] = 1
    
    #test
    if (test):
    
        f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(binary_output1, cmap='gray')
        ax1.set_title('RGB Thres (R)', fontsize=30)
        # -
        ax1_5.imshow(binary_output2, cmap='gray')
        ax1_5.set_title('RGB Thres (G)', fontsize=30)
        # -
        ax2.imshow(binary_output3, cmap='gray')
        ax2.set_title('RGB Thres (B)', fontsize=30)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    return binary_output

def luv_transform(img, thresh=(225, 255), test = False):

    l_channel = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)[:,:,0]
    
    l_thresh_min = 225
    l_thresh_max = 255
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh_min) & (l_channel <= l_thresh_max)] = 1
    
    #test
    if (test):
        u_channel = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)[:,:,1]
        u_binary = np.zeros_like(u_channel)
        u_binary[(u_channel >= thresh[0]) & (u_channel <= thresh[1])] = 1
        
        v_channel = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)[:,:,2]
        v_binary = np.zeros_like(v_channel)
        v_binary[(v_channel >= thresh[0]) & (v_channel <= thresh[1])] = 1
    
        f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(l_binary, cmap='gray')
        ax1.set_title('LUV Thres (L)', fontsize=30)
        # -
        ax1_5.imshow(u_binary, cmap='gray')
        ax1_5.set_title('LUV Thres (U)', fontsize=30)
        # -
        ax2.imshow(v_binary, cmap='gray')
        ax2.set_title('LUV Thres (V)', fontsize=30)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        
    return l_binary
    

def lab_transform(img, thresh=(155, 200), test = False):
    b_channel = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,2]   
    
    b_thresh_min = 155
    b_thresh_max = 200
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh_min) & (b_channel <= b_thresh_max)] = 1
    
    #test
    if (test):
        l_channel = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,0]
        l_binary = np.zeros_like(l_channel)
        l_binary[(l_channel >= thresh[0]) & (l_channel <= thresh[1])] = 1
        
        a_channel = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,1]
        a_binary = np.zeros_like(a_channel)
        a_binary[(a_channel >= thresh[0]) & (a_channel <= thresh[1])] = 1
    
        f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(l_binary, cmap='gray')
        ax1.set_title('LAB Thres (L)', fontsize=30)
        # -
        ax1_5.imshow(a_binary, cmap='gray')
        ax1_5.set_title('LAB Thres (A)', fontsize=30)
        # -
        ax2.imshow(b_binary, cmap='gray')
        ax2.set_title('LAB Thres (B)', fontsize=30)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    return b_binary

#combined transform for the image, using color and gradient
def thresh_transform(img, threshold_parameters='video', gradient_channel = False, hls_channels = [False,True,False], rgb_channels = [True,True,False], s_thresh=(170, 255), sx_thresh=(20, 100), test = False):
    img = np.copy(img)
    
    # Threshold color channel
    hls_img = hls_thresh_transform(img, channel = hls_channels, thresh=s_thresh, test = test)
    #rge transform
    rgb_img = rgb_transform(img, channel = rgb_channels, test = test)
    #gradient thresholds
    if (gradient_channel):
        gradient_img = sobel_thresh_transform(img)
    else:
        gradient_img = np.zeros_like(hls_img)
    
    #tests
    tst = luv_transform(img, test = test)
    tst2 = lab_transform(img, test = test)
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(gradient_img), gradient_img, hls_img)) * 255
    combined = np.zeros_like(gradient_img)
    combined[(gradient_img == 1) | (hls_img == 1) | (rgb_img ==1) ] = 1
    #show filters
    if (test):
        f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(hls_img, cmap='gray')
        ax1.set_title('HLS Thres', fontsize=30)
        # -
        ax1_5.imshow(gradient_img, cmap='gray')
        ax1_5.set_title('Gradient Thres', fontsize=30)
        # -
        ax2.imshow(combined, cmap='gray')
        ax2.set_title('Joint Thres', fontsize=30)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    return combined


## Lane pixel detection

In [5]:
# Define a class to receive the characteristics of each line detection
class LaneLine():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # how many times line has not been detected?
        self.not_detected_cnt = 0  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

In [70]:
def hist(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram


def measure_curvature_pixels(left_fit, right_fit, ploty):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def find_lane_pixels(binary_warped): 
    # ----- histogram -----
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # ----- sliding window -----
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero() #get non zero pixels from the image matrix (size 1028x1028)
    nonzeroy = np.array(nonzero[0]) # x-coords of non-zero pixels
    nonzerox = np.array(nonzero[1]) # y-coords of non-zero pixels
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin # Update this
        win_xleft_high = leftx_current + margin # Update this
        win_xright_low = rightx_current - margin # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        # good_left_inds / good_right_indx - stores indices in the nonzerox/nonzeroy arrays (which are also arrays of indices but in the image matrix)
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >= win_xright_low) & (nonzerox < win_xright_high) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))
        pass # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img, histogram


def draw_lane(binary_warped, left_fit, right_fit):
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    # draw the lines on empty image for output
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    return color_warp 

        
def fit_polynomial(binary_warped, test = False):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img, histogram = find_lane_pixels(binary_warped)
    
#     if (len(lefty) == 0) or (len(righty) == 0): #secure against no points given (possible if searched area was empty after all the filters)
#         return None, None, None, None, None

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    if (len(lefty) != 0) and (len(righty) != 0):
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty

        ## Visualization ##
        # Colors in the left and right lane regions
        if (test):
            out_img[lefty, leftx] = [255, 0, 0]
            out_img[righty, rightx] = [0, 0, 255]

            # Plots the left and right polynomials on the lane lines
        #     if (test):
    #         plt.plot(left_fitx, ploty, color='yellow')
    #         plt.plot(right_fitx, ploty, color='yellow')
            # ------
            histogram = histogram.astype('float32') / 255
            f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
            f.tight_layout()
            ax1.imshow(np.int_(np.logical_not(binary_warped)), cmap='gray')
            ax1.plot(binary_warped.shape[0]-histogram, color='red')
            ax1.set_title('Histogram', fontsize=30)
            # -
            ax1_5.imshow(out_img) #, cmap='gray'
            ax1_5.set_title('Pixel find with moving window', fontsize=30)
            # -
            ax2.imshow(out_img) #, cmap='gray'
            ax2.plot(left_fitx, ploty, color='yellow')
            ax2.plot(right_fitx, ploty, color='yellow')
            ax2.set_title('Fitted line', fontsize=30)
            plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

        # draw the lines on empty image for output

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        return color_warp, left_fit, right_fit, left_fitx, right_fitx, ploty
    else: #if no pixels detected at all
        if (test):
            # ------
            f, (ax1, ax1_5, ax2) = plt.subplots(1, 3, figsize=(24, 9))
            ax1.imshow(color_warp, cmap='gray')
            ax1.set_title('Histogram', fontsize=30)
            # -
            ax1_5.imshow(color_warp) #, cmap='gray'
            ax1_5.set_title('Pixel find with moving window', fontsize=30)
            # -
            ax2.imshow(color_warp) #, cmap='gray'
            ax2.set_title('Fitted line', fontsize=30)
            plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        return color_warp ,None, None, None, None, None

# ------------ fit if previous frame available -----------------

def fit_poly(img_shape, leftx, lefty, rightx, righty):
    if (len(lefty) == 0) or (len(righty) == 0): #secure against no points given (possible if searched area was empty after all the filters)
        return None, None, None, None, None
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit

def search_around_poly(binary_warped, prev_left_fit, prev_right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    
    #old
#     good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
#     good_right_inds = ((nonzerox >= win_xright_low) & (nonzerox < win_xright_high) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]

#     print("len(nonzero):"+str(len(nonzero))+ " ;len(nonzeroy):"+str(len(nonzeroy)) + " ;len(nonzerox):" + str(len(nonzerox)))
#     print("prev_left_fit:"+str(prev_left_fit))
#     print("prev_left_fit[0]:"+str(prev_left_fit[0])+ " ;prev_left_fit[1]:"+str(prev_left_fit[1]) + " ;prev_left_fit[2]:" + str(prev_left_fit[2]) )
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (prev_left_fit[0]*(nonzeroy**2) + prev_left_fit[1]*nonzeroy + 
                    prev_left_fit[2] - margin)) & (nonzerox < (prev_left_fit[0]*(nonzeroy**2) + 
                    prev_left_fit[1]*nonzeroy + prev_left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (prev_right_fit[0]*(nonzeroy**2) + prev_right_fit[1]*nonzeroy + 
                    prev_right_fit[2] - margin)) & (nonzerox < (prev_right_fit[0]*(nonzeroy**2) + 
                    prev_right_fit[1]*nonzeroy + prev_right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty, left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    if left_fitx is not None:
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
        # Plot the polynomial lines onto the image
    #     plt.plot(left_fitx, ploty, color='yellow')
    #     plt.plot(right_fitx, ploty, color='yellow')
        ## End visualization steps ##

        # draw the lines on empty image for output

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
#    return result
    return color_warp, left_fit, right_fit, left_fitx, right_fitx, ploty

#return avg line distnace or -1 if distance changes abruptly at some section 
def line_distance(left_fit, right_fit, ploty, xm_per_pix = 3.7/700):
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    diff_fitx =  right_fitx - left_fitx
    #print(diff_fitx)
    mean_x = np.mean(diff_fitx)
    std_x = np.std(diff_fitx)
    z_score = (diff_fitx - mean_x) / std_x
    z_score_max = np.max(z_score)
    #print("avg[m]: "+str(mean_x*xm_per_pix)+", std:"+str(std_x),", max_z:"+str(z_score_max)+", std/mean:"+str(std_x/mean_x)+", top[m]:"+str(diff_fitx[0]*xm_per_pix)+", bottom[m]:"+str(diff_fitx[len(diff_fitx)-1]*xm_per_pix))
    #print(z_score)
    if (std_x/mean_x) >= 0.1: #look for abnormal standard deviation distance between lanes (== non parallel)
        return -1
    elif ((mean_x*xm_per_pix)> max_lane_width) or ((mean_x*xm_per_pix)< min_lane_width) : # check for abnormal lane width in meters
        return -2
    elif ( diff_fitx[len(diff_fitx)-1]*xm_per_pix > bottom_max_lane_width) or ( diff_fitx[len(diff_fitx)-1]*xm_per_pix < bottom_min_lane_width):
        return -3
    elif ( diff_fitx[0]*xm_per_pix > bottom_max_lane_width) or ( diff_fitx[0]*xm_per_pix < bottom_min_lane_width):
        return -3
    else:
        return np.mean(diff_fitx)

def lane_line_sanity_check(new_left_fit, new_right_fit, lane_left, lane_right, ploty, xm_per_pix = 3.7/700):
    if (new_left_fit is None) or (new_right_fit is None):
        return -5
    #1. check if lines have similar curvature
    #left_curverad_m, right_curverad_m = measure_curvature_real(new_left_fit, new_right_fit, ploty)
    left_curverad, right_curverad = measure_curvature_pixels(new_left_fit, new_right_fit, ploty)
    
#     print("curveture[m], l:"+str(left_curverad*xm_per_pix)+ ", r:"+str(right_curverad*xm_per_pix)+ ", l-r:"+str((left_curverad-right_curverad)*xm_per_pix) ) 
#     print("curveture[m_v2], l:"+str(left_curverad_m)+ ", r:"+str(right_curverad_m)+ ", l-r:"+str((left_curverad_m-right_curverad_m)) ) 
    #print("curveture[px], l:"+str(left_curverad)+ ", r:"+str(right_curverad)+ ", l-r:"+str((left_curverad-right_curverad))+ ", l-r/l:"+str( ((left_curverad-right_curverad)/left_curverad)) + ", l-r/r:"+str( ((left_curverad-right_curverad)/right_curverad)) ) 
    #2+3: 
    # - check if lines are separated by approximately the right distance horizontally
    # - check if lines are roughly parallel
#     print("parallel check:")
    # -- left
    a_left = new_left_fit[0]
    b_left = new_left_fit[1]
    c_left = new_left_fit[2]
    # -- right
    a_right = new_right_fit[0]
    b_right = new_right_fit[1]
    c_right = new_right_fit[2]
    # calc diff
    a_diff = a_left - a_right
    b_diff = b_left - b_right
    c_diff = c_left - c_right
    #curverure difference measured as difference in function parameters
    #print("a_diff: "+str(a_diff)+" - b_diff: "+str(b_diff)+" - c_diff: "+str(c_diff) )
#     print("a_diff(l): "+str(a_diff/a_left)+" - b_diff(l): "+str(b_diff/b_left)+" - c_diff(l): "+str(c_diff/c_left) )
#     print("a_diff(r): "+str(a_diff/a_right)+" - b_diff(r): "+str(b_diff/b_right)+" - c_diff(r): "+str(c_diff/c_right) )
#     if (a_diff/a_left) > a_thres or (b_diff/b_left) > a_thres or (c_diff/c_left) > a_thres:
        
#         return -4
#     if (a_diff/a_right) > a_thres or (b_diff/b_right) > a_thres or (c_diff/c_right) > a_thres:
#         return -5
    
    #parallel check v2 (check distance for all pixels, look at mean/std/z-score)
    #print("dist check:")
    lane_dist = line_distance(new_left_fit, new_right_fit, ploty)
    if  lane_dist < 0:
        return lane_dist
    else:
        return 1

def fit_polynomial_multiframe(binary_warped, lane_left = None , lane_right = None, test = False):
    
#     #test
#     output, left_fit, right_fit, ploty = fit_polynomial(binary_warped, test = test)
#     #write to Lane objects
#     if (lane_left is None):
#         lane_left = LaneLine()
#     if (lane_right is None):
#         lane_right = LaneLine()

#     lane_left.current_fit = left_fit
#     lane_right.current_fit = right_fit
#     test_sanity = lane_line_sanity_check(left_fit, right_fit, lane_left, lane_right, ploty)
#     return  output, lane_left, lane_right, ploty, test_sanity
    
    #print(str(binary_warped.shape[0])+"x"+str(binary_warped.shape[1]))
    #get lane lines
    
    if (lane_left is None or lane_right is None): #if previous not available do histogram
        output, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(binary_warped, test = test)
        #write to Lane objects
        if (lane_left is None):
            lane_left = LaneLine()
        if (lane_right is None):
            lane_right = LaneLine()
    else: #if previous available search around the polynomial calculated in prior frame
        output, left_fit, right_fit, left_fitx, right_fitx, ploty = search_around_poly(binary_warped, prev_left_fit = lane_left.current_fit , prev_right_fit = lane_right.current_fit )
        if (left_fit is not None):
            test_sanity = lane_line_sanity_check(left_fit, right_fit, lane_left, lane_right, ploty) 
            if (test_sanity<0) and ( (lane_left.not_detected_cnt >= not_detected_cnt_thres) or (lane_right.not_detected_cnt >= not_detected_cnt_thres)):
                output, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(binary_warped, test = test)
        else:
            output, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(binary_warped, test = test)
        #check if lines correct, if not do histogram
    
    #do some best effort checks if lines are correct
    
    #if not correct use prior frame polynomial
            
    test_sanity = lane_line_sanity_check(left_fit, right_fit, lane_left, lane_right, ploty)

    #update curve only if lane detected correctly (or if first frame), otherwise fall back to old calculation
#     print("test_sanity:"+str(test_sanity))
#     print("left_fit:")
#     print(left_fit)
#     if (lane_left is not None):
#         print("left_fit_curr:")
#         print(lane_left.current_fit)
    if (test_sanity>=0) or (len(lane_left.current_fit)<=1) or ((lane_left.not_detected_cnt >= not_detected_cnt_max_thres) and (left_fit is not None) and (right_fit is not None)): 
        lane_left.current_fit = left_fit
        lane_left.allx = left_fitx
        lane_left.ally = ploty
        
        lane_right.current_fit = right_fit
        lane_right.allx = right_fitx
        lane_right.ally = ploty
        
        #zero detection counter
        lane_left.not_detected_cnt = 0
        lane_right.not_detected_cnt = 0
    else:
        #print("using prior curve!!")
        #mark that lanes were not detected
        lane_left.not_detected_cnt += 1
        lane_right.not_detected_cnt += 1
#     if (not test_sanity):
#         print("##")
#         cv2.rectangle(output,(100,100), (50,50),(255,0,0), -1) 

    output = draw_lane(binary_warped, lane_left.current_fit, lane_right.current_fit)
#     print(lane_left.allx)
#     print(lane_left.allx)
    return output, lane_left, lane_right, test_sanity #, ploty
    


## Curvature of the lane and vehicle position

In [96]:
def measure_lane_values_real(image, lane_left, lane_right):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    left_fit = lane_left.current_fit
    right_fit = lane_right.current_fit
    ploty = lane_left.ally
    leftx = lane_left.allx
    rightx = lane_right.allx
    
    if (left_fit is None) or (right_fit is None): #error handling
        return 0,0
        
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/1280 #720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    #ploty, left_fit_cr, right_fit_cr = generate_data(ym_per_pix, xm_per_pix)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    #calculate car position
    mid_image = image.shape[1]//2 # image centre
    car_position = (leftx[-1] + rightx[-1])/2 # Car position
    car_offset = (mid_image - car_position) * xm_per_pix # horizontal offset 
    
    #plot onto image
    output = image.copy()
    cv2.putText(output, 'Left lane line curvature: {:.2f} m'.format(left_curverad), 
                (60, 60), cv2.FONT_HERSHEY_PLAIN, 1.5, (255,255,255), 2)
    cv2.putText(output, 'Right lane line curvature: {:.2f} m'.format(right_curverad), 
                (60, 90), cv2.FONT_HERSHEY_PLAIN, 1.5, (255,255,255), 2)
    cv2.putText(output, 'Horizontal car offset: {:.2f} m'.format(car_offset), 
                (60, 120), cv2.FONT_HERSHEY_PLAIN, 1.5, (255,255,255), 2) #FONT_HERSHEY_SIMPLEX
    
    
    return output, left_curverad, right_curverad

# Pipeline

In [90]:
# Define a class to store both lanes and pass to pipeline for updating
class LaneLines():
    def __init__(self):
        self.update_data = False # flag if data should be updated based on new frames or not
        #left/ right lane data
        self.lane_left = None  
        self.lane_right = None
    


# image - image to analyse and annotate
# mtx, dist - parameters for correcting camera distortion
# lane_left - left lane data if available from previous frames
# lane_right - right lane data if available from previous frames
def extract_lane_boundries(image,mtx, dist, lane_data = LaneLines(), test = False):
    image_org = image.copy()
    output = image.copy()
    # 2. Apply a distortion correction to raw images.
    undistorted_img = cv2.undistort(image, mtx, dist, None, mtx)
    if (test):
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        # ax1.imshow(output2, cmap='gray')
        ax1.imshow(image_org)
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(undistorted_img, cmap='gray')
        ax2.set_title('Undistorted Image', fontsize=30)
#        cv2.imwrite("examples/writeup/undistorted_img_ex.jpg", undistorted_img)
    output = undistorted_img.copy()
    # 2.5 apply blur
    output = gaussian_blur(output, kernel=3)
    # 3. Use color transforms, gradients, etc., to create a thresholded binary image.
    #output = sobel_thresh_transform(output)
    output = thresh_transform(output, gradient_channel = gradient_channel_param, hls_channels = hls_channels_param, rgb_channels = rgb_channels_param, test = test)
    # 4. Apply a perspective transform to rectify binary image ("birds-eye view").
    output, M, Minv = warp(output, image_color = image_org, warp_source_coords = warp_source_coordinates, warp_dest_coords = warp_destination_coordinates, test = test)
    # 5. Detect lane pixels and fit to find the lane boundary.
    output, lane_left, lane_right,tst = fit_polynomial_multiframe(output, lane_left = lane_data.lane_left , lane_right = lane_data.lane_right, test = test) #, ploty
    # 6. Warp the detected lane boundaries back onto the original image.
    output = unwarp(undistorted_img, output, Minv)
    # 7. Determine the curvature of the lane and vehicle position with respect to center.
    output, left_curverad, right_curverad = measure_lane_values_real(output, lane_left, lane_right) #
    # 8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
    #print(left_curverad, right_curverad)
    if ((tst<0) & test==True):
        if (tst == -3): #bad bottom width
            cv2.rectangle(output,(100,100), (50,50),(0,0,255), -1) 
        elif (tst == -2): #bad width
            cv2.rectangle(output,(100,100), (50,50),(0,255,0), -1) 
        else: #non-parallel
            cv2.rectangle(output,(100,100), (50,50),(255,0,0), -1) 
    
    if (lane_data.update_data == True):
        lane_data.lane_left = lane_left
        lane_data.lane_right = lane_right
    
    return output #, lane_left, lane_right


# TEST

### Test still frames

In [91]:
verbose_mode = False
#challenge vid coords
warp_source_coordinates = [[240, 720] , [580, 490], [740, 490], [1125, 720]]
warp_destination_coordinates = [[250,  720], [250,    0],  [1065,   0], [1065, 720]]
not_detected_cnt_max_thres = 8
hls_channels_param = [False, True,False] #use S channel
rgb_channels_param = [True, True, False] #use non of RGB
gradient_channel_param = False


#regular video coords
# warp_source_coordinates = [[150, 720] , [570, 460], [710, 460], [1175, 720]]
# warp_destination_coordinates = [[250,  720], [250,    0],  [1065,   0], [1065, 720]]
# not_detected_cnt_max_thres = 100
# hls_channels_param = [False, False,True] #use S channel
# rgb_channels_param = [False, False, False] #use non of RGB
# gradient_channel_param = True

#1. prep - calibrate cammera

# Make a list of calibration images
camera_calib_images = glob.glob('camera_cal/calibration*.jpg')
mtx, dist = calibrateCammera(camera_calib_images, test = verbose_mode)

#2. prep - read-in data
# test_images = sorted(glob.glob('test_video3/*.jpg'))
test_images = sorted(glob.glob('challenge_video/*.jpg'))
# test_images = sorted(glob.glob('harder_challenge_video/*.jpg'))


#3. run pipeline for all images

#lane_left = None
#lane_right = None
lane_data = LaneLines()
lane_data.update_data = True

for idx, fpath in enumerate(test_images):
    print('Processing '+fpath)
#     if (fpath == "challenge_video/challenge_video-00130.jpg"):
#         verbose_mode = True
    img = cv2.imread(fpath)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    img_src = fpath #test_images[3]
    img = mpimg.imread(img_src)
    #, lane_left, lane_right
    img_out = extract_lane_boundries(img, mtx, dist, lane_data = lane_data, test = verbose_mode) #
    
    if (verbose_mode):
        #visualise result
        # Visualize
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        # ax1.imshow(output2, cmap='gray')
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(img_out, cmap='gray')
        ax2.set_title('Processed Image', fontsize=30)

    #4. write output
    filename =os.path.basename(fpath)
    filename_noext = os.path.splitext(filename)[0]
#     write_name = "harder_challenge_video_out/"+filename_noext+".jpg"
    write_name = "challenge_video_out/"+filename_noext+".jpg"
#     write_name = "test_video_out3/"+filename_noext+".jpg"
    cv2.imwrite(write_name, cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR))  

Processing test_video3/project_video-00001.jpg
Processing test_video3/project_video-00002.jpg
Processing test_video3/project_video-00003.jpg
Processing test_video3/project_video-00004.jpg
Processing test_video3/project_video-00005.jpg
Processing test_video3/project_video-00006.jpg
Processing test_video3/project_video-00007.jpg
Processing test_video3/project_video-00008.jpg
Processing test_video3/project_video-00009.jpg
Processing test_video3/project_video-00010.jpg
Processing test_video3/project_video-00011.jpg
Processing test_video3/project_video-00012.jpg
Processing test_video3/project_video-00013.jpg
Processing test_video3/project_video-00014.jpg
Processing test_video3/project_video-00015.jpg
Processing test_video3/project_video-00016.jpg
Processing test_video3/project_video-00017.jpg
Processing test_video3/project_video-00018.jpg
Processing test_video3/project_video-00019.jpg
Processing test_video3/project_video-00020.jpg
Processing test_video3/project_video-00021.jpg
Processing te

Processing test_video3/project_video-00176.jpg
Processing test_video3/project_video-00177.jpg
Processing test_video3/project_video-00178.jpg
Processing test_video3/project_video-00179.jpg
Processing test_video3/project_video-00180.jpg
Processing test_video3/project_video-00181.jpg
Processing test_video3/project_video-00182.jpg
Processing test_video3/project_video-00183.jpg
Processing test_video3/project_video-00184.jpg
Processing test_video3/project_video-00185.jpg
Processing test_video3/project_video-00186.jpg
Processing test_video3/project_video-00187.jpg
Processing test_video3/project_video-00188.jpg
Processing test_video3/project_video-00189.jpg
Processing test_video3/project_video-00190.jpg
Processing test_video3/project_video-00191.jpg
Processing test_video3/project_video-00192.jpg
Processing test_video3/project_video-00193.jpg
Processing test_video3/project_video-00194.jpg
Processing test_video3/project_video-00195.jpg
Processing test_video3/project_video-00196.jpg
Processing te

Processing test_video3/project_video-00351.jpg
Processing test_video3/project_video-00352.jpg
Processing test_video3/project_video-00353.jpg
Processing test_video3/project_video-00354.jpg
Processing test_video3/project_video-00355.jpg
Processing test_video3/project_video-00356.jpg
Processing test_video3/project_video-00357.jpg
Processing test_video3/project_video-00358.jpg
Processing test_video3/project_video-00359.jpg
Processing test_video3/project_video-00360.jpg
Processing test_video3/project_video-00361.jpg
Processing test_video3/project_video-00362.jpg
Processing test_video3/project_video-00363.jpg
Processing test_video3/project_video-00364.jpg
Processing test_video3/project_video-00365.jpg
Processing test_video3/project_video-00366.jpg
Processing test_video3/project_video-00367.jpg
Processing test_video3/project_video-00368.jpg
Processing test_video3/project_video-00369.jpg
Processing test_video3/project_video-00370.jpg
Processing test_video3/project_video-00371.jpg
Processing te

Processing test_video3/project_video-00526.jpg
Processing test_video3/project_video-00527.jpg
Processing test_video3/project_video-00528.jpg
Processing test_video3/project_video-00529.jpg
Processing test_video3/project_video-00530.jpg
Processing test_video3/project_video-00531.jpg
Processing test_video3/project_video-00532.jpg
Processing test_video3/project_video-00533.jpg
Processing test_video3/project_video-00534.jpg
Processing test_video3/project_video-00535.jpg
Processing test_video3/project_video-00536.jpg
Processing test_video3/project_video-00537.jpg
Processing test_video3/project_video-00538.jpg
Processing test_video3/project_video-00539.jpg
Processing test_video3/project_video-00540.jpg
Processing test_video3/project_video-00541.jpg
Processing test_video3/project_video-00542.jpg
Processing test_video3/project_video-00543.jpg
Processing test_video3/project_video-00544.jpg
Processing test_video3/project_video-00545.jpg
Processing test_video3/project_video-00546.jpg
Processing te

Processing test_video3/project_video-00701.jpg
Processing test_video3/project_video-00702.jpg
Processing test_video3/project_video-00703.jpg
Processing test_video3/project_video-00704.jpg
Processing test_video3/project_video-00705.jpg
Processing test_video3/project_video-00706.jpg
Processing test_video3/project_video-00707.jpg
Processing test_video3/project_video-00708.jpg
Processing test_video3/project_video-00709.jpg
Processing test_video3/project_video-00710.jpg
Processing test_video3/project_video-00711.jpg
Processing test_video3/project_video-00712.jpg
Processing test_video3/project_video-00713.jpg
Processing test_video3/project_video-00714.jpg
Processing test_video3/project_video-00715.jpg
Processing test_video3/project_video-00716.jpg
Processing test_video3/project_video-00717.jpg
Processing test_video3/project_video-00718.jpg
Processing test_video3/project_video-00719.jpg
Processing test_video3/project_video-00720.jpg
Processing test_video3/project_video-00721.jpg
Processing te

Processing test_video3/project_video-00876.jpg
Processing test_video3/project_video-00877.jpg
Processing test_video3/project_video-00878.jpg
Processing test_video3/project_video-00879.jpg
Processing test_video3/project_video-00880.jpg
Processing test_video3/project_video-00881.jpg
Processing test_video3/project_video-00882.jpg
Processing test_video3/project_video-00883.jpg
Processing test_video3/project_video-00884.jpg
Processing test_video3/project_video-00885.jpg
Processing test_video3/project_video-00886.jpg
Processing test_video3/project_video-00887.jpg
Processing test_video3/project_video-00888.jpg
Processing test_video3/project_video-00889.jpg
Processing test_video3/project_video-00890.jpg
Processing test_video3/project_video-00891.jpg
Processing test_video3/project_video-00892.jpg
Processing test_video3/project_video-00893.jpg
Processing test_video3/project_video-00894.jpg
Processing test_video3/project_video-00895.jpg
Processing test_video3/project_video-00896.jpg
Processing te

Processing test_video3/project_video-01051.jpg
Processing test_video3/project_video-01052.jpg
Processing test_video3/project_video-01053.jpg
Processing test_video3/project_video-01054.jpg
Processing test_video3/project_video-01055.jpg
Processing test_video3/project_video-01056.jpg
Processing test_video3/project_video-01057.jpg
Processing test_video3/project_video-01058.jpg
Processing test_video3/project_video-01059.jpg
Processing test_video3/project_video-01060.jpg
Processing test_video3/project_video-01061.jpg
Processing test_video3/project_video-01062.jpg
Processing test_video3/project_video-01063.jpg
Processing test_video3/project_video-01064.jpg
Processing test_video3/project_video-01065.jpg
Processing test_video3/project_video-01066.jpg
Processing test_video3/project_video-01067.jpg
Processing test_video3/project_video-01068.jpg
Processing test_video3/project_video-01069.jpg
Processing test_video3/project_video-01070.jpg
Processing test_video3/project_video-01071.jpg
Processing te

Processing test_video3/project_video-01226.jpg
Processing test_video3/project_video-01227.jpg
Processing test_video3/project_video-01228.jpg
Processing test_video3/project_video-01229.jpg
Processing test_video3/project_video-01230.jpg
Processing test_video3/project_video-01231.jpg
Processing test_video3/project_video-01232.jpg
Processing test_video3/project_video-01233.jpg
Processing test_video3/project_video-01234.jpg
Processing test_video3/project_video-01235.jpg
Processing test_video3/project_video-01236.jpg
Processing test_video3/project_video-01237.jpg
Processing test_video3/project_video-01238.jpg
Processing test_video3/project_video-01239.jpg
Processing test_video3/project_video-01240.jpg
Processing test_video3/project_video-01241.jpg
Processing test_video3/project_video-01242.jpg
Processing test_video3/project_video-01243.jpg
Processing test_video3/project_video-01244.jpg
Processing test_video3/project_video-01245.jpg
Processing test_video3/project_video-01246.jpg
Processing te

### Harder challanging Video test (TODO)

In [75]:
verbose_mode = False

#calibrate cammera
camera_calib_images = glob.glob('camera_cal/calibration*.jpg')
mtx, dist = calibrateCammera(camera_calib_images, test = verbose_mode)

#init lane data
lane_data = LaneLines()
lane_data.update_data = True
#pipeline settings specific for challenge video
warp_source_coordinates = [[240, 720] , [580, 490], [740, 490], [1125, 720]]
warp_destination_coordinates = [[250,  720], [250,    0],  [1065,   0], [1065, 720]]
not_detected_cnt_max_thres = 8

#process video
output_video_path = 'harder_challenge_video_annotated.mp4'
#white_output = 'solidWhiteRight666.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
input_video = VideoFileClip("harder_challenge_video.mp4")
output_video = input_video.fl_image(lambda image: process_image(image, mtx, dist, lane_data)) #NOTE: this function expects color images!!
#white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time output_video.write_videofile(output_video_path, audio=False) #, codec='libvpx'

t:   0%|          | 2/1199 [00:00<01:51, 10.76it/s, now=None]

Moviepy - Building video harder_challenge_video_annotated.mp4.
Moviepy - Writing video harder_challenge_video_annotated.mp4



Moviepy - Done !
Moviepy - video ready harder_challenge_video_annotated.mp4
CPU times: user 15min 24s, sys: 1min 7s, total: 16min 32s
Wall time: 4min 56s
